In [18]:
import pyspark
import pandas as pd

from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F

In [19]:
pyspark.__file__

'/home/gianluca/spark/spark-3.5.0-bin-hadoop3/python/pyspark/__init__.py'

In [20]:
spark.version

'3.5.0'

In [ ]:
# import raw data
# !wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
# !wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-10.csv.gz

In [ ]:
# check raw data 
# !head data/taxi+_zone_lookup.csv

In [ ]:
# write output data as parquet
# df.write.parquet('data/zones')

In [ ]:
# df_fhv = spark.read \
#     .option("header", "true") \
#     .csv('fhv_tripdata_2019-10.csv.gz')

In [ ]:
# convert data type using pandas

# df_pandas = pd.read_csv('fhv_tripdata_2019-10.csv.gz')

# df_pandas.dtypes

In [ ]:
# CREATE SCHEMA FOR OUR PARQUET FILE

# fhv_schema = types.StructType([
#     types.StructField('dispatching_base_num', types.StringType(), True),
#     types.StructField('pickup_datetime', types.TimestampType(), True),
#     types.StructField('dropOff_datetime', types.TimestampType(), True),
#     types.StructField('PUlocationID', types.IntegerType(), True),
#     types.StructField('DOLocationID', types.IntegerType(), True),
#     types.StructField('SR_Flag', types.StringType(), True),
#     types.StructField('Affiliated_base_number', types.StringType(), True),
# ])

In [ ]:
# df_fhv = spark.read \
#     .option("header", "true") \
#     .schema(fhv_schema) \
#     .csv('fhv_tripdata_2019-10.csv.gz')

In [ ]:
# df_fhv.show()

In [ ]:
# df_fhv.printSchema()

In [ ]:
# Repartition the fhv Dataframe to 6 partitions and save it to parquet.
# df_fhv = df_fhv.repartition(6)

In [ ]:
# df_fhv.write.parquet('data/fhv/2019/10/')

What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

Answer = 6MB

In [3]:
# spark gcp configuration
credentials_location = '/home/gianluca/notebooks/gcp_credentials.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "/home/gianluca/lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [4]:
# spark gcp context
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

24/02/24 20:17:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
# spark gcp create session
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [6]:
df_fhv_pq = spark.read.parquet('gs://dtc_data_lake_de-zoomcamp-hw5/data/fhv/*/*/*')

In [7]:
df_fhv_pq.show(2)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02784|2019-10-01 09:55:38|2019-10-01 10:05:43|          89|          85|   NULL|                  NULL|
|              B02429|2019-10-21 04:15:47|2019-10-21 04:36:04|         264|         264|   NULL|                B02429|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
only showing top 2 rows



In [8]:
df_fhv_pq.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [9]:
# create SQL temp table to start writing SQL queries on fhv data
df_fhv_pq.createOrReplaceTempView('fhv_oct_trips_data')

In [10]:
# Question 3:
# Count records

# How many taxi trips were there on the 15th of October?

spark.sql("""
    SELECT 
        date_trunc('day', pickup_datetime) AS pickup_date,
        count(*) as trip_count
    FROM fhv_oct_trips_data
    WHERE date_trunc('day', pickup_datetime) = '2019-10-15 00:00:00'
    GROUP BY 1
""").show()

+-------------------+----------+
|        pickup_date|trip_count|
+-------------------+----------+
|2019-10-15 00:00:00|     62610|
+-------------------+----------+



In [11]:
# Question 4:
# Longest trip for each day

# What is the length of the longest trip in the dataset in hours?

spark.sql("""
    SELECT 
        MAX(DATEDIFF(hour, pickup_datetime, dropOff_datetime)) as max_hours_difference
    FROM fhv_oct_trips_data
""").show()

+--------------------+
|max_hours_difference|
+--------------------+
|              631152|
+--------------------+



In [12]:
# Question 5:
# User Interface

# Spark’s User Interface which shows the application's dashboard runs on which local port? 4040

Question 6:
Least frequent pickup location zone

Load the zone lookup data into a temp view in Spark
Zone Data

Using the zone lookup data and the FHV October 2019 data, what is the name of the LEAST frequent pickup location Zone?

In [13]:
df_zone_lookup = spark.read.parquet('gs://dtc_data_lake_de-zoomcamp-hw5/data/zones/*')

df_zone_lookup.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [14]:
# join zone look up with the fhv trips data to answer question no. 6
df_join = df_fhv_pq.join(df_zone_lookup, df_fhv_pq.PUlocationID == df_zone_lookup.LocationID, how='inner')

In [15]:
df_join.createOrReplaceTempView('fhv_joined_zones_data')

In [16]:
spark.sql("""
    SELECT 
        Zone,
        count(*) as zone_pickup_count
    FROM fhv_joined_zones_data
    WHERE Zone in ('East Chelsea', 'Jamaica Bay', 'Union Sq', 'Crown Heights North')
    GROUP BY Zone
""").show()

+-------------------+-----------------+
|               Zone|zone_pickup_count|
+-------------------+-----------------+
|Crown Heights North|             3036|
|           Union Sq|             1893|
|       East Chelsea|             1381|
|        Jamaica Bay|                1|
+-------------------+-----------------+

